In [11]:
import os, sys, shutil, zipfile, csv
import requests
import pandas as pd
import numpy as np
import geopandas
from sqlalchemy import URL, create_engine, text

# Read in GTFS

In [15]:
def import_bundle(
    bundle,
    import_trips=True,
    import_shapes=True,
    import_stops=True,
    import_stoptimes=True,
    import_calendar=True,
    import_calendar_dates=True,
    cache_bundle=False,
):
    """Import GTFS data from the following tables in ADLS:
    - core.dbo.fact_gtfs_trips
    - core.dbo.fact_gtfs_shapes
    - core.dbo.fact_gtfs_stops
    - core.dbo.fact_gtfs_shape_reference
    - core.dbo.fact_gtfs_stop_times

    """

    # GTFS bundle can take some time to import. If there is already a cached GTFS bundle pickle file in the repository, read in that data instead
    picklefile = bundle + ".pickle"
    if os.path.isfile(picklefile) == True:
        with open(picklefile, "rb") as handle:
            gtfs_bundle = pickle.load(handle)
            return gtfs_bundle

    con = create_engine(
        r"trino://trino-route-trino.apps.mtasiprod.eastus.aroapp.io:443/mtadatalake",
        connect_args={
            "auth": OAuth2Authentication(),
            "http_scheme": "https",
        }
    )
    cur = con.connect()
    
    output = {}
    # Import trips
    trip_sql = f"""
    with agency as ( -- get agency id for each route
        select distinct route_id, agency_id from mtadatalake.core.fact_gtfs_routes 
        where bundle = '{bundle}'
        )
    SELECT trips.route_id ,trip_id, service_id,trip_headsign,direction_id,block_id,shape_id,boarding_type,bundle, agency.agency_id
    FROM mtadatalake.core.fact_gtfs_trips trips
    join agency on trips.route_id = agency.route_id
    where trips.bundle = '{bundle}'
    """
    if import_trips == True:
        print('Loading trips')
        f = cur.execute(text(trip_sql))
        trips_df = pd.DataFrame(f.fetchall())
        output["trips"] = trips_df

    # Import shapes
    shape_sql = f"""
    SELECT shape_id, shape_pt_sequence, shape_pt_lat, shape_pt_lon, bundle
    FROM mtadatalake.core.fact_gtfs_shapes
    where bundle = '{bundle}'
    """
    if import_shapes == True:
        print("Loading shapes")
        f = cur.execute(text(shape_sql))
        shapes_df = pd.DataFrame(f.fetchall())
        output["shapes"] = shapes_df

    # Import stops
    stop_sql = f"""
    SELECT fact_gtfs_stops.stop_id, stop_name, stop_lat, stop_lon, shape_ref.revenue_stop, bundle
    FROM mtadatalake.core.fact_gtfs_stops
    left join (
        SELECT stop_id, MAX(revenue_stop) AS revenue_stop
        FROM mtadatalake.core.fact_gtfs_shape_reference
        where bundle = '{bundle}'
        GROUP BY stop_id
    ) shape_ref
    on fact_gtfs_stops.stop_id = shape_ref.stop_id
    where bundle = '{bundle}'
    """
    if import_stops == True:
        print("Loading stops")
        f = cur.execute(text(stop_sql))
        stops_df = pd.DataFrame(f.fetchall())
        output["stops"] = stops_df

    # Import stop_times
    stoptime_sql = f"""
    SELECT trip_id, stop_id, arrival_time, departure_time, timepoint, stop_sequence, pickup_type, drop_off_type, bundle
    FROM mtadatalake.core.fact_gtfs_stop_times
    where bundle = '{bundle}'
    """
    if import_stoptimes == True:
        print("Loading stop times")
        f = cur.execute(text(stoptime_sql))
        stoptimes_df = pd.DataFrame(f.fetchall())
        output["stoptimes"] = stoptimes_df

    # import calendar
    calendar_sql = f""" 
    SELECT service_id, monday, tuesday, wednesday, thursday, friday, saturday, sunday, start_date, end_date, bundle, modified_time, loaded_time
    FROM mtadatalake.core.fact_gtfs_calendar
    where bundle = '{bundle}'
    """
    if import_calendar == True:
        print("Loading calendar")
        f = cur.execute(text(calendar_sql))
        calendar_df = pd.DataFrame(f.fetchall())
        output["calendar"] = calendar_df

    #import calendar_dates
    calendar_dates_sql = f"""
    SELECT service_id, "date", exception_type
    FROM mtadatalake.core.fact_gtfs_calendar_dates
    where bundle = '{bundle}'
    """
    if import_calendar_dates == True:
        print("Loading calendar_dates")
        f = cur.execute(text(calendar_dates_sql))
        calendar_dates_df = pd.DataFrame(f.fetchall())
        output["calendar_dates"] = calendar_dates_df

    
    if cache_bundle == True:
        with open(picklefile, "wb") as handle:
            pickle.dump(output, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print(f"GTFS data for {bundle} successfully loaded and cached")
    else:
       print(f"GTFS data for {bundle} successfully loaded") 

    return output

In [3]:
def find_most_representative(bundle1,bundle2):
    trino = dbhook.Hook(adls_trino)

    representative_day_query = f'''
    with bundle_dates AS (-- Get a list of dates and bundles
      SELECT service_date, format_datetime(service_date, 'EEEE') day_of_week, bundle, pick_year, pick_name, sched_type, manual
      FROM mtadatalake.core.dim_bus_gtfs_bundle_dates
      where bundle in ('{bundle1}', '{bundle2}')
      --  where pick_year in (2024) 
    )
    , gtfs_calendar_fixed_dates as (
      -- Correct invalid dates like February 31st
      select service_id, monday, tuesday, wednesday, thursday, friday, saturday, sunday, bundle
        , start_date/10000 start_yr, start_date/100 % 100 start_mnth
        -- Take out dates like Feb 31 or Apr 31
        , case 
          when start_date/100 % 100 = 2 and start_date % 100 > 28 then 28 
          when (start_date/10000) % 4 = 0 and start_date % 100 > 29 then 29 -- if it's a leap year, 2/29
          when start_date/100 % 100 in (9, 4, 6, 11) and start_date % 100 > 30 then 30
          else start_date % 100
        end as start_day
        , end_date/10000 end_yr, end_date/100 % 100 end_mnth
        , case
          when end_date/100 % 100 = 2 and end_date % 100 > 28 then 28
          when (end_date/10000) % 4 = 0 and end_date % 100 > 29 then 29 -- if it's a leap year, 2/29
          when end_date/100 % 100 in (9, 4, 6, 11) and end_date % 100 > 30 then 30
          else end_date % 100
        end as end_day
      from mtadatalake.core.fact_gtfs_calendar
      where bundle in (select distinct bundle from bundle_dates)
    )
    , calendar as (
      -- convert start_date to date object
      select service_id, monday, tuesday, wednesday, thursday, friday, saturday, sunday, bundle
        , DATE(cast(start_yr AS VARCHAR) || '-' || CAST(start_mnth AS VARCHAR) || '-' || CAST(start_day AS VARCHAR)) as start_date
        , DATE(CAST(end_yr AS VARCHAR) || '-' || CAST(end_mnth AS VARCHAR) || '-' || CAST(end_day AS VARCHAR)) AS end_date
      from gtfs_calendar_fixed_dates
    )
    , exceptions as (
      SELECT service_id, "date", exception_type, bundle
        , DATE(cast(("date" / 10000) as varchar) || '-' || cast((("date" / 100) % 100) as varchar) || '-' || cast(("date" % 100) as varchar)) AS service_date
        , format_datetime(DATE(cast(("date" / 10000) as varchar) || '-' || cast((("date" / 100) % 100) as varchar) || '-' || cast(("date" % 100) as varchar)), 'EEEE') as day_of_week
      FROM mtadatalake.core.fact_gtfs_calendar_dates
      where bundle in (select distinct bundle from bundle_dates)
    )
    , base_schedule as (
      -- for each day, assemble a row for each service id serving it
      SELECT bd.service_date, bd.day_of_week, c.service_id, c.bundle, e.exception_type
      FROM bundle_dates bd
      inner join calendar c
        ON bd.bundle = c.bundle 
        and bd.service_date BETWEEN c.start_date AND c.end_date
        AND (
          (bd.day_of_week = 'Monday' AND c.monday = 1) OR
          (bd.day_of_week = 'Tuesday' AND c.tuesday = 1) OR
          (bd.day_of_week = 'Wednesday' AND c.wednesday = 1) OR
          (bd.day_of_week = 'Thursday' AND c.thursday = 1) OR
          (bd.day_of_week = 'Friday' AND c.friday = 1) OR
          (bd.day_of_week = 'Saturday' AND c.saturday = 1) OR
          (bd.day_of_week = 'Sunday' AND c.sunday = 1)
          )
      -- Join in dates and service_ids where service was removed (exception_type 2)
      left join (select * from exceptions where exception_type = 2) e
        on c.bundle = e.bundle
        and bd.service_date = e.service_date
        and c.service_id = e.service_id
    )
    , modified_schedules as (
      -- Join in dates and service_ids where service was added (exception_type 1)
      select service_date, day_of_week, service_id, bundle, exception_type
      from base_schedule 
      where exception_type is null or exception_type = 1
      union all
      select service_date, day_of_week, service_id, bundle, exception_type
      from exceptions 
      where exception_type = 1
    )
    , daily_schedules as (
      -- Designate schedule daytype
      select service_date, day_of_week, service_id, bundle
        , CASE 
            WHEN day_of_week(service_date) BETWEEN 1 AND 5 THEN 'Weekday'
            WHEN day_of_week(service_date) = 6 THEN 'Saturday'
            WHEN day_of_week(service_date) = 7 THEN 'Sunday'
            ELSE null
          end as sched_daytype
      from modified_schedules
    )
    , schedules_per_day AS (
        -- Aggregate service_ids for each day to form a "schedule"
        SELECT 
            bundle, service_date, sched_daytype
            , array_join(array_agg(CAST(service_id AS VARCHAR) ORDER BY service_id), ',') AS schedule
        FROM daily_schedules
        GROUP BY bundle, service_date, sched_daytype
    )
    , schedule_variations AS (
      -- Count the occurrences of each unique schedule
      SELECT bundle, service_date, schedule, sched_daytype
        , COUNT(service_date) over (partition by bundle, schedule, sched_daytype) AS sched_var_frequency
        -- Make a name for each unique schedule, e.g. Weekday-1, Saturday-3
        ,  sched_daytype || '-' || CAST(DENSE_RANK() OVER (PARTITION BY bundle, sched_daytype ORDER BY schedule) AS VARCHAR) AS schedule_variation
      FROM schedules_per_day
    )
    , schedule_variations_ranked as ( -- this CTE returns one row per day, with the schedule variation and rank
      select bundle, service_date, schedule, sched_daytype, schedule_variation, sched_var_frequency
        , row_number() over (partition by bundle, sched_daytype order by sched_var_frequency desc, service_date) ranking_by_freq
      from schedule_variations
    )
    , most_representative_day as ( -- this CTE will give a weekday, saturday, and sunday date for each bundle
      -- When ranking_by_freq = 1, that day is the best representation of a weekday, saturday, or sunday schedule
        select bundle, service_date, sched_daytype as day_of_week
        from schedule_variations_ranked
        where ranking_by_freq = 1
    )
    -- decide between schedule_variations_ranked, most_representative_day, or sched_with_service_id
    select * from most_representative_day
    '''
    return trino.frame(representative_day_query)

### Loading in Historical GTFS

In [100]:
gtfshistorical = import_bundle('2025June_Prod_r04_b01_PREDATE_SHUTTLES_SCHEDULED')

Loading trips
Loading shapes
Loading stops
Loading stop times
Loading calendar
Loading calendar_dates
GTFS data for 2025June_Prod_r04_b01_PREDATE_SHUTTLES_SCHEDULED successfully loaded


In [103]:
trips_h = gtfshistorical['trips']
stoptimes_h = gtfshistorical['stoptimes']
stops_h = gtfshistorical['stops']
calendar_h = gtfshistorical['calendar']
calendar_dates_h = gtfshistorical['calendar_dates']

### Constructing schedule

In [104]:
# define date, day_of_week
targetdates = [20250702,20250705,20250706]
days_of_week = ['wednesday','saturday','sunday']
schedules = []
#looping through each date
for i in range(3):
    ### get service_ids from fact_gtfs_calendar
    ## where service_date is between start and end, and day of week being the same
    base_service_ids = calendar_h[(calendar_h[days_of_week[i]] == 1) & (calendar_h['start_date'] <= targetdates[i]) & (calendar_h['end_date'] >= targetdates[i])]['service_id']
    # Join in service_ids where service was removed (exception_type 2) from calendar_dates
    # Join in service_ids where service was added (exception_type 1) from calendar_dates
    exceptions = calendar_dates_h[calendar_dates_h['date'] == targetdates[i]]
    removed = exceptions[exceptions['exception_type'] == 2]['service_id']
    added = exceptions[exceptions['exception_type'] == 1]['service_id']
    
    #not including type 2
    baseminusremoved = base_service_ids[~base_service_ids.isin(removed)]
    
    #including type 1
    finalservices = pd.concat([baseminusremoved,added]).drop_duplicates()

    # for each service_id, get trips associated
    # finalservices becomes a df
    tripstoday = trips_h.merge(finalservices.to_frame(name='service_id'), on='service_id', how='inner')

    # for each of those trips, get stop_times 
    stoptimestoday = stoptimes_h.merge(tripstoday[['route_id','direction_id','trip_id']], on='trip_id', how='inner')
    
    ### Calculating Average (or Median) Running Time and Frequency
    # creating hour and minute manually from arrival_time
    stoptimestoday[['hour', 'minute','seconds']] = stoptimestoday['arrival_time'].str.split(':', expand=True).astype(int)
    stoptimestoday['time_in_minutes'] = stoptimestoday['hour'] * 60 + stoptimestoday['minute']

    ## creating origin and destination
    stoptimestoday['stop_sequence'] = stoptimestoday['stop_sequence'].astype(int)
    
    # Group by trip_id
    groupedbytrip = stoptimestoday.groupby('trip_id')['stop_sequence']

    # Origin = 1 if stop_sequence is the minimum in the trip
    stoptimestoday['origin'] = (stoptimestoday['stop_sequence'] == groupedbytrip.transform('min')).astype(int)

    # Destination = 1 if stop_sequence is the maximum in the trip
    stoptimestoday['destination'] = (stoptimestoday['stop_sequence'] == groupedbytrip.transform('max')).astype(int)
    stoptimestoday['hour'] = stoptimestoday['hour'] % 24

    # Filtering to only starts and ends
    STTfiltered = stoptimestoday[((stoptimestoday['origin']==1)| (stoptimestoday['destination'] ==1))]

    # Get start time from origin rows
    origin = STTfiltered[STTfiltered['origin'] == 1][['trip_id', 'route_id','direction_id','hour','time_in_minutes']]
    origin = origin.rename(columns={'time_in_minutes': 'start_min'})
    
    # Get end time from destination rows
    dest = STTfiltered[STTfiltered['destination'] == 1][['trip_id', 'time_in_minutes']]
    dest = dest.rename(columns={'time_in_minutes': 'end_min'})
    
    # Merge origin and destination rows on trip_id
    trip_summary = pd.merge(origin, dest, on='trip_id')
    
    # Duration in minutes
    trip_summary['duration_min'] = trip_summary['end_min'] - trip_summary['start_min']

    # Remove trips that start before 0 or end at or after 1440
    trip_summary = trip_summary[(trip_summary['start_min'] >= 0) & (trip_summary['end_min'] < 1440)]

    # Final summary: group by route and hour
    result = trip_summary.groupby(['route_id','direction_id','hour']).agg(
        frequency=('trip_id', 'count'),
        avg_running_time=('duration_min', 'median')
    ).reset_index().sort_values(['route_id','direction_id', 'hour'])

    # Flagging those that have considerable difference between the mean and median
    threshold = 1
    result['Flag'] = abs(result['avg_running_time'] - trip_summary.groupby(['route_id','direction_id','hour'])['duration_min'].mean().values) > threshold

    # Create a complete grid of route_id, direction_id, and hours 0-23
    all_hours = pd.DataFrame({'hour': list(range(24))})
    routes = trip_summary[['route_id', 'direction_id']].drop_duplicates()
    full_grid = routes.merge(all_hours, how='cross')
    
    # Merge result with full_grid to ensure every hour appears
    result = full_grid.merge(result, on=['route_id', 'direction_id', 'hour'], how='left')
    
    # Fill Day and Date for all rows
    result['Day'] = days_of_week[i]
    result['Date'] = targetdates[i]
    result = result.sort_values(by=['route_id', 'direction_id', 'hour'])
    
    schedules.append(result)

finalschedule1 = pd.concat(schedules)


In [105]:
gtfshistorical = import_bundle('2025March_Prod_r01_b05_SHUTTLES_PREDATE_SCHEDULED')
trips_h = gtfshistorical['trips']
stoptimes_h = gtfshistorical['stoptimes']
stops_h = gtfshistorical['stops']
calendar_h = gtfshistorical['calendar']
calendar_dates_h = gtfshistorical['calendar_dates']

Loading trips
Loading shapes
Loading stops
Loading stop times
Loading calendar
Loading calendar_dates
GTFS data for 2025March_Prod_r01_b05_SHUTTLES_PREDATE_SCHEDULED successfully loaded


In [106]:
# define date, day_of_week
targetdates = [20250401,20250405,20250406]
days_of_week = ['tuesday','saturday','sunday']
schedules = []
#looping through each date
for i in range(3):
    ### get service_ids from fact_gtfs_calendar
    ## where service_date is between start and end, and day of week being the same
    base_service_ids = calendar_h[(calendar_h[days_of_week[i]] == 1) & (calendar_h['start_date'] <= targetdates[i]) & (calendar_h['end_date'] >= targetdates[i])]['service_id']
    # Join in service_ids where service was removed (exception_type 2) from calendar_dates
    # Join in service_ids where service was added (exception_type 1) from calendar_dates
    exceptions = calendar_dates_h[calendar_dates_h['date'] == targetdates[i]]
    removed = exceptions[exceptions['exception_type'] == 2]['service_id']
    added = exceptions[exceptions['exception_type'] == 1]['service_id']
    
    #not including type 2
    baseminusremoved = base_service_ids[~base_service_ids.isin(removed)]
    
    #including type 1
    finalservices = pd.concat([baseminusremoved,added]).drop_duplicates()

    # for each service_id, get trips associated (via merge)
    # finalservices becomes a df
    tripstoday = trips_h.merge(finalservices.to_frame(name='service_id'), on='service_id', how='inner')

    # for each of those trips, get stop_times (via merge)
    stoptimestoday = stoptimes_h.merge(tripstoday[['route_id','direction_id','trip_id']], on='trip_id', how='inner')
    
    ### Calculating Average (or Median) Running Time and Frequency
    # creating hour and minute manually from arrival_time
    stoptimestoday[['hour', 'minute','seconds']] = stoptimestoday['arrival_time'].str.split(':', expand=True).astype(int)
    stoptimestoday['time_in_minutes'] = stoptimestoday['hour'] * 60 + stoptimestoday['minute']

    ## creating origin and destination
    stoptimestoday['stop_sequence'] = stoptimestoday['stop_sequence'].astype(int)
    
    # Group by trip_id
    groupedbytrip = stoptimestoday.groupby('trip_id')['stop_sequence']

    # Origin = 1 if stop_sequence is the minimum in the trip
    stoptimestoday['origin'] = (stoptimestoday['stop_sequence'] == groupedbytrip.transform('min')).astype(int)

    # Destination = 1 if stop_sequence is the maximum in the trip
    stoptimestoday['destination'] = (stoptimestoday['stop_sequence'] == groupedbytrip.transform('max')).astype(int)
    stoptimestoday['hour'] = stoptimestoday['hour'] % 24

    # Filtering to only starts and ends
    STTfiltered = stoptimestoday[((stoptimestoday['origin']==1)| (stoptimestoday['destination'] ==1))]

    # Get start time from origin rows
    origin = STTfiltered[STTfiltered['origin'] == 1][['trip_id', 'route_id','direction_id','hour','time_in_minutes']]
    origin = origin.rename(columns={'time_in_minutes': 'start_min'})
    
    # Get end time from destination rows
    dest = STTfiltered[STTfiltered['destination'] == 1][['trip_id', 'time_in_minutes']]
    dest = dest.rename(columns={'time_in_minutes': 'end_min'})
    
    # Merge origin and destination rows on trip_id
    trip_summary = pd.merge(origin, dest, on='trip_id')
    
    # Duration in minutes
    trip_summary['duration_min'] = trip_summary['end_min'] - trip_summary['start_min']

    # Remove trips that start before 0 or end at or after 1440
    trip_summary = trip_summary[(trip_summary['start_min'] >= 0) & (trip_summary['end_min'] < 1440)]

    # Final summary: group by route and hour
    result = trip_summary.groupby(['route_id','direction_id','hour']).agg(
        frequency=('trip_id', 'count'),
        avg_running_time=('duration_min', 'median')
    ).reset_index().sort_values(['route_id','direction_id', 'hour'])

    # Flagging those that have considerable difference between the mean and median
    threshold = 1
    result['Flag'] = abs(result['avg_running_time'] - trip_summary.groupby(['route_id','direction_id','hour'])['duration_min'].mean().values) > threshold

    # Create a complete grid of route_id, direction_id, and hours 0-23
    all_hours = pd.DataFrame({'hour': list(range(24))})
    routes = trip_summary[['route_id', 'direction_id']].drop_duplicates()
    full_grid = routes.merge(all_hours, how='cross')
    
    # Merge result with full_grid to ensure every hour appears
    result = full_grid.merge(result, on=['route_id', 'direction_id', 'hour'], how='left')
    
    # Fill Day and Date for all rows
    result['Day'] = days_of_week[i]
    result['Date'] = targetdates[i]
    result = result.sort_values(by=['route_id', 'direction_id', 'hour'])
    
    schedules.append(result)

finalschedule2 = pd.concat(schedules)

In [107]:
merged = pd.merge(
    finalschedule1,
    finalschedule2,
    on=["route_id", "direction_id", "hour", "Day"],
    how="outer"
)
merged = merged.sort_values(by=["Day","route_id", "direction_id", "hour"])

# Move 'Day' to the front
cols = merged.columns.tolist()
cols.remove("Day")
cols = ["Day"] + cols
merged = merged[cols]


# Save the merged result
merged.to_csv("final_schedule.csv", index=False)